In [1]:
from pyDeepInsight import ImageTransformer
from pyDeepInsight.utils import Norm2Scaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import TSNE
import pandas as pd
import numpy as np
from umap import UMAP
import os

c:\Users\Mello\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def min_max_scaling(data):
    min_val = min(data)
    max_val = max(data)
    normalized_data = [(x - min_val) / (max_val - min_val) for x in data]
    return normalized_data

def process_npy_file(file_path):
    # Load the npy file
    data = np.load(file_path, allow_pickle=True)
    frames = []
    class_label = data[0]
    frames.append(class_label)
    for i in range(1, len(data[1])):
        data[1][i] = min_max_scaling(data[1][i])
        frames+=(data[1][i])
    print(len(frames))

    row_data = {'ClassLabel': class_label}
    frame_index = 1 # Start from 1 to skip the class_label
    for j in range(1, 20):
        for i in range(0, 66):
            if frame_index + 2 < len(frames): # Ensure we have at least 3 elements left
                row_data[f'X{i}{j}'] = frames[frame_index]
                row_data[f'Y{i}{j}'] = frames[frame_index + 1]
                row_data[f'Z{i}{j}'] = frames[frame_index + 2]
                frame_index += 3 # Move to the next set of 3 elements
            else:
                print(f"Not enough elements left in frames for index {frame_index}")
                break
        if frame_index + 2 >= len(frames):
            print("Reached the end of frames")
            break
    return row_data

In [3]:
folder_path = 'dataset_generator\dataset'

columns_part = []
for j in range(1, 20):
    for i in range(0, 66):
        temp = [f'X{i}{j}'] + [f'Y{i}{j}'] + [f'Z{i}{j}']
        columns_part += temp

columns = ['ClassLabel'] + columns_part
df = pd.DataFrame(columns = columns)

# Iterate over .npy files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.npy'):
        file_path = os.path.join(folder_path, file_name)

        # Process the npy file and get row data
        row_data = process_npy_file(file_path)

        # Append the row to the DataFrame
        df.loc[len(df)] = row_data

# df.to_json('dataset.json', orient = 'split', compression = 'infer', index = 'true')
df.head()

3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3565
Reached the end of frames
3565
Reached the end of frames
3565
Reached the end of frames
3565
Reached the end of frames
3565
Reached the end of frames
3565
Reached the end of frames
3565
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Reached the end of frames
3763
Rea

In [ ]:
y = df['ClassLabel'].values
X = df.iloc[:, 1:].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y)

In [ ]:
len(np.unique(y, return_counts=True)[0])

17

In [ ]:
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)
num_classes = np.unique(y_train_enc).size

In [ ]:
print("Original X_train size:", X_train.size)
print("Expected size after reshaping:", 577 * 198 * 19 * 1)

# Add this line to check the dimensions of X_train before reshaping
print("X_train dimensions before reshaping:", X_train.shape)

Original X_train size: 723558
Expected size after reshaping: 2170674
X_train dimensions before reshaping: (577, 1254)


In [ ]:
# Model Traiining using CNN

import numpy as np
import pickle
import cv2, os
from glob import glob
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras import backend as K
K.set_image_data_format('channels_first')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

def get_num_of_classes():
	return len(np.unique(y, return_counts=True)[0])

coords = 198
frames = 19

def cnn_model():
	num_of_classes = get_num_of_classes()
	model = Sequential()
	model.add(Conv2D(16, (2,2), input_shape=(frames, 1), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
	model.add(Conv2D(32, (3,3), activation='relu'))
	model.add(MaxPooling2D(pool_size=(3, 3), strides=(3, 3), padding='same'))
	model.add(Conv2D(64, (5,5), activation='relu'))
	model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(num_of_classes, activation='softmax'))
	sgd = optimizers.SGD(lr=1e-2)
	model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
	filepath="cnn_model_keras2.h5"
	checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
	callbacks_list = [checkpoint1]
	#from keras.utils import plot_model
	#plot_model(model, to_file='model.png', show_shapes=True)
	return model, callbacks_list

def train(X_train, X_test, y_train_enc, y_test_enc):
	X_train = X_train.reshape(X_train.shape[0], frames, 1).astype('float32')
	X_test = X_test.reshape(X_test.shape[0], frames, 1).astype('float32')

	model, callbacks_list = cnn_model()
	model.summary()
	model.fit(X_train, y_train_enc, validation_data=(X_test, y_test_enc), epochs=15, batch_size=500, callbacks=callbacks_list)
	scores = model.evaluate(X_test, y_test_enc, verbose=0)
	print("CNN Error: %.2f%%" % (100-scores[1]*100))
	#model.save('cnn_model_keras2.h5')

train(X_train, X_test, y_train_enc, y_test_enc)
K.clear_session();


ValueError: cannot reshape array of size 723558 into shape (577,19,1)